## Get additional EarthCARE products

Configure everything in config.toml, in particular my credentials

Note that the script is outputing a lot of information in stdout, because it's designed to be run e.g in VSCode (i.e not with a notebook). I had to customize the oads_download.py script to prevent outputing too much information

TODO: Keep only data within a 250-km radius.

In [1]:
# https://earthcarehandbook.earth.esa.int/catalogue/index

In [2]:
# Used in Galfione et al.:
# C-PRO: Reflectivity and doppler velocity... does not exist: C-FMR contains the corrected reflectivity, and C_CD the corrected doppler velocity
# C-CD:  Sedimentation velocity best estimate
# C-FMR: Regions contaminated by multiple scattering (mask)

# Others:
# CPR_CLD_2A: Providing cloud water content and characteristic diameters, liquid water path, and ice water path.

# MSI_COP_2A: For 2D overview

# Conclusions:
# Download CPR_CD for sedimentation velocity and doppler velocity, CPR-CLD for liquid/ice water paths and sedimentation estimates, and C-FMR for corrected reflectity and  masking;
# Download also ACM_CAP (experimental)
# Download MSI_COP_2A to talk to Blanka and Johanna

In [3]:
# General
import os
import glob
import subprocess
import pickle
import xarray as xr
from tqdm.notebook import tqdm

In [4]:
PTHS = {
    # 'sar_pth': '/home/arthur/results/sar_extended/05_addPRIMED/dsg.nc',
    'ear_dir': '/home/arthur/results/TCsLifeMonitFromObs/PRJCT_VERTICAL_WIND/01_COLLOCATE_EARTHCARE_TCs/AC__TC__2B/',
    # 'sav_dir': '/home/arthur/results/TCsLifeMonitFromObs/PRJCT_VERTICAL_WIND/02_EARTHCARE_DIAGNOSTIC/'
}

# Open EarthCARE
all_data = glob.glob(PTHS['ear_dir'] + '*.nc')
all_data.sort()

In [5]:
threshold          = 250000 # meters, exclude if EC's closest_distance_to_track_center_ibt is greater than
list_of_overpasses = []

for file in tqdm(all_data):
    # Open file
    # ds = xr.open_dataset(file)
    ds      = xr.open_dataset(file)
    if ds.closest_distance_to_track_center_ibt <= threshold:
        list_of_overpasses.append(os.path.basename(file).split('.')[0].split('_')[-1])

    # Close
    ds.close()

  0%|          | 0/418 [00:00<?, ?it/s]

In [6]:
# ### Eventually save the list of files as list of EC overpasses

# list_of_overpasses = [os.path.basename(file).split('.')[0].split('_')[-1] for file in all_data]

# # To save the list of frames
# filename           = PTHS['ear_dir'] + 'list_of_overpasses.pkl'
# with open(filename, 'wb') as file:  # 'wb' pour écriture binaire
#     pickle.dump(list_of_overpasses, file)

In [7]:
# # Or, to load it
# filename = '/home/arthur/results/TCsLifeMonitFromObs/PRJCT_VERTICAL_WIND/02_EARTHCARE_DIAGNOSTIC/' + 'list_of_overpasses.pkl'
# with open(filename, 'rb') as file:  # 'rb' pour lecture binaire
#     oaf_values = pickle.load(file)

In [8]:
# # Test
# list_of_overpasses = ['01708B', '02089A', '02089B', '02618E', '02691A', '02698E', '03958E', '04237E', '04237F', '04290A']

In [9]:
# # Test
# all_data = ['/home/arthur/results/TCsLifeMonitFromObs/PRJCT_VERTICAL_WIND/01_COLLOCATE_EARTHCARE_TCs/AC__TC__2B/ECA_EXAA_AC__TC__2B_20240915T172841Z_20240915T232259Z_01708B.nc']

In [1]:
# Try this
# In brief, I need to apply the script oads_download.py to each overpass, and for each requested product.

for oaf in tqdm(list_of_overpasses):
    # cmd = ["python", "oads_download.py", "CPR_CD__2A", "-oaf", oaf]
    # cmd = ["python", "oads_download.py", "CPR_CLD_2A", "-oaf", oaf]
    # cmd = ["python", "oads_download.py", "CPR_FMR_2A", "-oaf", oaf]
    # cmd = ["python", "oads_download.py", "MSI_COP_2A", "-oaf", oaf]
    cmd = ["python", "oads_download.py", "ACMCAP", "-oaf", oaf]
    print(f"Running command: {' '.join(cmd)}")
    subprocess.run(cmd)